In [142]:
import pymysql
import pandas.io.sql as psql
%matplotlib inline
from __future__ import print_function

try:
    xrange
except NameError:
    xrange = range

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as sp
import scipy.sparse as sparse
from sklearn.decomposition import NMF, TruncatedSVD #非負値、特異値
import time

In [108]:
con = pymysql.connect(host="recommendation-201810.csbsmnjyxb52.ap-northeast-1.rds.amazonaws.com",
                     user="guest1",
                     password="guest1",
                      database="movies",
                     port=3306)

In [109]:
sql = """
SELECT
a.userId,
a.movieId,
c.title,
a.rating
FROM rating as a 
INNER JOIN 
links as b
on a.movieId = b.movieId
INNER JOIN
movies_meta as c 
ON b.tmdbid = c.tmdbid
"""

In [140]:
data = psql.read_sql(sql, con)
movie = data.drop_duplicates(subset=['userId', 'title'])
movie_sel = movie[['movieId', 'title']].drop_duplicates()
movie_dict = dict(zip(movie_sel['movieId'].astype(int), movie_sel['title']))
                             
rating_matrix = \
 movie.pivot(index='title', columns='userId', values='rating')
rating_matrix.fillna(0, inplace=True)
                             
user_list = np.array(rating_matrix.columns)
title_list = np.array(rating_matrix.index)
                             
rating_matrix_ar = np.array(rating_matrix)
                             
def get_cosine_similarity(x, y):
    return 1- sp.distance.cosine(x, y)

def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

In [153]:
t1 = time.time() 

movie_id = 101

movie_similarity = []
target_movie_row = rating_matrix_ar[list(rating_matrix.index).index(movie_dict[movie_id])]
for row in rating_matrix_ar:
    sim = get_jaccard_similarity(target_movie_row, row)
    movie_similarity.append(sim)
movie_similarity = np.array(movie_similarity)

topN = 20
idx = movie_similarity.argsort()[::-1][1:topN+1]
selected_movie_similarity = movie_similarity[idx]
selected_rating = rating_matrix_ar[idx]
                             
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_movie_similarity)
    similarity_sum = \
     sum(selected_movie_similarity[selected_movie_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in title_list[avg_score.argsort()[::-1]]:
    #if recommended_topic not in already_followed_topic:
    print(recommended_topic)
    counter += 1
    if recommend_num <= counter:
        break

t2 = time.time()
 
# 経過時間を表示
elapsed_time = t2-t1
print(f"----------------------------")
print(f"経過時間：{elapsed_time}")

Away from Her
22 Jump Street
America's Sweethearts
Angels in America
Agnes of God
----------------------------
経過時間：0.6812129020690918


In [154]:
svd = TruncatedSVD(n_components=20)
rating_matrix_sparse = sparse.lil_matrix(rating_matrix_ar)
rating_matrix_svd = svd.fit_transform(rating_matrix_sparse)

In [155]:
t1 = time.time() 

movie_id = 101

movie_similarity = []
target_movie_row = rating_matrix_svd[list(rating_matrix.index).index(movie_dict[movie_id])]
for row in rating_matrix_svd:
    sim = get_jaccard_similarity(target_movie_row, row)
    movie_similarity.append(sim)
movie_similarity = np.array(movie_similarity)

topN = 20
idx = movie_similarity.argsort()[::-1][1:topN+1]
selected_movie_similarity = movie_similarity[idx]
selected_rating = rating_matrix_ar[idx]
                             
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_movie_similarity)
    similarity_sum = \
     sum(selected_movie_similarity[selected_movie_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in title_list[avg_score.argsort()[::-1]]:
    #if recommended_topic not in already_followed_topic:
    print(recommended_topic)
    counter += 1
    if recommend_num <= counter:
        break

t2 = time.time()
 
# 経過時間を表示
elapsed_time = t2-t1
print(f"----------------------------")
print(f"経過時間：{elapsed_time}")

Back in the Day
A Single Man
A Simple Twist of Fate
A Simple Plan
A Shot in the Dark
----------------------------
経過時間：0.5588192939758301


C:\Users\Hiroshi.Hiromax\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [156]:
nmf = NMF(n_components=10, random_state=1236, init='random')
rating_matrix_sparse = sparse.lil_matrix(rating_matrix_ar)
rating_matrix_nmf = nmf.fit_transform(rating_matrix_sparse)

In [157]:
t1 = time.time() 

movie_id = 101

movie_similarity = []
target_movie_row = rating_matrix_nmf[list(rating_matrix.index).index(movie_dict[movie_id])]
for row in rating_matrix_nmf:
    sim = get_jaccard_similarity(target_movie_row, row)
    movie_similarity.append(sim)
movie_similarity = np.array(movie_similarity)

topN = 20
idx = movie_similarity.argsort()[::-1][1:topN+1]
selected_movie_similarity = movie_similarity[idx]
selected_rating = rating_matrix_ar[idx]
                             
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_movie_similarity)
    similarity_sum = \
     sum(selected_movie_similarity[selected_movie_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

recommend_num = 5
counter = 0
for recommended_topic in title_list[avg_score.argsort()[::-1]]:
    #if recommended_topic not in already_followed_topic:
    print(recommended_topic)
    counter += 1
    if recommend_num <= counter:
        break

t2 = time.time()
 
# 経過時間を表示
elapsed_time = t2-t1
print(f"----------------------------")
print(f"経過時間：{elapsed_time}")

Back in the Day
A Single Man
A Simple Twist of Fate
A Simple Plan
A Shot in the Dark
----------------------------
経過時間：0.5406308174133301


C:\Users\Hiroshi.Hiromax\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


### 2. https://jsonlint.com/ でなぜエラーが出るのかを確認する